In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import classification_report, confusion_matrix
from skimage.feature import hog

: 

In [ ]:
# Paths to datasets
jaffe_path = "drive/MyDrive/jaffedbase"  # JAFFE dataset path
ckplus_path = "drive/MyDrive/CK+"        # CK+ dataset path

In [ ]:
# Emotion mappings
jaffe_emotion_map = {
    "AN": 0, "DI": 1, "FE": 2, "HA": 3, "NE": 4, "SA": 5, "SU": 6
}
ckplus_emotion_map = {
    "anger": 0, "disgust": 1, "fear": 2, "happy": 3, "sadness": 5, "surprise": 6
}

In [ ]:
# Load JAFFE dataset
def load_jaffe_dataset(dataset_path, emotion_map, target_size=(128, 128)):
    images, labels = [], []
    for img_file in os.listdir(dataset_path):
        img_path = os.path.join(dataset_path, img_file)
        if os.path.isfile(img_path) and img_file.endswith('.tiff'):
            emotion_code = img_file.split('.')[1][:2]
            if emotion_code in emotion_map:
                label = emotion_map[emotion_code]
                img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
                if img is not None:
                    img_resized = cv2.resize(img, target_size)
                    images.append(img_resized / 255.0)  # Normalize
                    labels.append(label)
    return np.array(images), np.array(labels)

In [ ]:
# Load CK+ dataset
def load_ckplus_dataset(dataset_path, emotion_map, target_size=(128, 128)):
    images, labels = [], []
    for emotion, label in emotion_map.items():
        emotion_folder = os.path.join(dataset_path, emotion)
        if os.path.exists(emotion_folder):
            for img_file in os.listdir(emotion_folder):
                img_path = os.path.join(emotion_folder, img_file)
                if img_file.endswith(('.png', '.jpg', '.jpeg', '.tiff')):
                    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
                    if img is not None:
                        img_resized = cv2.resize(img, target_size)
                        images.append(img_resized / 255.0)  # Normalize
                        labels.append(label)
    return np.array(images), np.array(labels)

In [ ]:
# Load datasets
jaffe_images, jaffe_labels = load_jaffe_dataset(jaffe_path, jaffe_emotion_map)
ckplus_images, ckplus_labels = load_ckplus_dataset(ckplus_path, ckplus_emotion_map)

In [ ]:
# Combine datasets
combined_images = np.concatenate([jaffe_images, ckplus_images])
combined_labels = np.concatenate([jaffe_labels, ckplus_labels])

In [ ]:
# Flatten images for feature extraction
flat_images = combined_images.reshape(combined_images.shape[0], -1)

In [ ]:
# Train-test split
X_train, X_temp, y_train, y_temp = train_test_split(flat_images, combined_labels, test_size=0.3, stratify=combined_labels, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, stratify=y_temp, random_state=42)

In [ ]:
# Feature extraction using HOG
def extract_hog_features(images):
    hog_features = []
    for img in images:
        features = hog(img.reshape(128, 128), pixels_per_cell=(8, 8), cells_per_block=(2, 2), orientations=9, block_norm='L2-Hys')
        hog_features.append(features)
    return np.array(hog_features)

X_train_hog = extract_hog_features(X_train)
X_val_hog = extract_hog_features(X_val)
X_test_hog = extract_hog_features(X_test)

In [ ]:
# Models
models = {
    "SVM": SVC(kernel='linear', C=1.0, random_state=42),
    "KNN": KNeighborsClassifier(n_neighbors=5),
    "Decision Tree": DecisionTreeClassifier(random_state=42),
    "Naive Bayes": GaussianNB()
}

In [ ]:
# Train and evaluate models
def train_and_evaluate_model(model, model_name, X_train, y_train, X_val, y_val):
    print(f"\nTraining {model_name}...")
    model.fit(X_train, y_train)
    y_val_pred = model.predict(X_val)

    # Validation metrics
    print(f"\nValidation Classification Report for {model_name}:")
    print(classification_report(y_val, y_val_pred))

    # Confusion matrix
    conf_matrix = confusion_matrix(y_val, y_val_pred)
    plt.figure(figsize=(8, 6))
    sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues',
                xticklabels=jaffe_emotion_map.keys(), yticklabels=jaffe_emotion_map.keys())
    plt.title(f"Confusion Matrix - {model_name}")
    plt.xlabel("Predicted")
    plt.ylabel("True")
    plt.show()

    return model

trained_models = {}
for model_name, model in models.items():
    trained_models[model_name] = train_and_evaluate_model(
        model, model_name, X_train_hog, y_train, X_val_hog, y_val
    )

In [ ]:
# Test models
def test_model(model, model_name, X_test_hog, y_test, X_test_original):
    print(f"\nTesting {model_name}...")
    # Predict on test data
    y_test_pred = model.predict(X_test_hog)

    # Visualization: Show 3 test images in a row with predictions
    example_indices = [0, 1, 2]  # Indices of example images to visualize
    fig, axes = plt.subplots(1, len(example_indices), figsize=(12, 4))  # 1 row, 3 columns
    fig.suptitle(f"{model_name} Predictions", fontsize=16)

    for ax, idx in zip(axes, example_indices):
        ax.imshow(X_test_original[idx].reshape(128, 128), cmap='gray')
        ax.set_title(f"True: {y_test[idx]}\nPred: {y_test_pred[idx]}")
        ax.axis('off')

    plt.tight_layout()
    plt.show()

In [ ]:
# Test each trained model
for model_name, model in trained_models.items():
    test_model(model, model_name, X_test_hog, y_test, X_test)